## Data characterization


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, re
import seaborn as sns

Read in the station list "../Data/trim.slst"

In [2]:
slst = "../Data/trim.slst"
sta=np.loadtxt(slst,dtype='str')
sta_num=len(sta)
print("number of station: ",sta_num,"\nStation: ",sta)


number of station:  30 
Station:  ['BGU' 'BRPU' 'BSUT' 'CTU' 'CVRU' 'DUG' 'FOR1' 'FORU' 'HMU' 'HVU1' 'HVU2'
 'HWUT' 'JLU' 'LCMT' 'LIUT' 'MPU' 'MTPU' 'NLU' 'NOQ' 'PNSU' 'PSUT' 'RDMU'
 'SPU' 'SRU' 'SWUT' 'SZCU' 'TCRU' 'TCU' 'VRUT1' 'VRUT2']


Data visualization with histograms and xy-plot. Output the normalized correlation and mean values of each component. 

In [ ]:
usecols=["utvec", "dv", "err", "temp", "SM_EWT", "snow_EWT", "GSL", "UL", "date"]
root = "../Data/"

file_loc = open("../Data/sta.loc", "r")

fdv="Data_cc.csv"
file_dv = open(fdv, "w")
content="stan,lon,lat,elv,cc_dv_dv,cc_dv_temp,cc_dv_soil,cc_dv_snow,cc_dv_GSL,cc_dv_UL"
file_dv.write(content+"\n")

fmean="Data_mean.csv"
file_mean = open(fmean, "w")
content="stan,lon,lat,elv,mean_dv,mean_temp,mean_soil,mean_snow,mean_GSL,mean_UL"
file_mean.write(content+"\n")

for sk in range(sta_num):
    stnm=sta[sk].strip()
    for line in file_loc:
        if re.search(stnm, line):
            loc=line.strip()
            break
    
    fn = root+"INTERP_"+stnm+".csv"
    fi = pd.read_csv(fn,names=usecols,header=0)
    fi['date'] = fi['date'].astype(str)
    dtime=np.array(fi.utvec)
    
    data=np.zeros((6,len(np.array(fi.dv))))
    data[0,:]=np.array(fi.dv)
    data[1,:]=np.array(fi.temp)    
    data[2,:]=np.array(fi.SM_EWT)  
    data[3,:]=np.array(fi.snow_EWT)
    data[4,:]=np.array(fi.GSL)     
    data[5,:]=np.array(fi.UL)      
    
    figname="Data_hist_"+stnm+".png"
    labels=["dvv","temp","soil","snow","GSL","UL"]
    fig,ax =plt.subplots(6,6,figsize=(20,16))
    for k in range(6):
        ax[k,0].hist(data[k,:]  , bins=20, label=labels[k],edgecolor="white")
        ax[k,0].axvline(x=np.mean(data[k,:]), color='r', label="Mean")
        ax[k,0].set_ylabel("Count")
        ax[k,0].set_title("Data: "+labels[k])
        ax[k,0].legend()

        for j in range(6):
            if (k>j):
                ax[j,k].set_xlabel(labels[j])
                ax[j,k].set_ylabel(labels[k])
                ax[j,k].set_title(labels[j]+" and "+labels[k])
                ax[j,k].plot(data[j,:],data[k,:],marker='.',markersize=1,linestyle = 'None')
    
    ax[0,0].set_xlabel("dv/v (%)")
    ax[1,0].set_xlabel("Temperature (degree C)")
    ax[2,0].set_xlabel("Equivalent water thickness (m)")
    ax[3,0].set_xlabel("Equivalent water thickness (m)")
    ax[4,0].set_xlabel("water level (m)")
    ax[5,0].set_xlabel("water level (m)")

    plt.tight_layout()
    plt.savefig(figname)
    
    cc=np.zeros(shape=(6))
    mean=np.zeros(shape=(6))
    for k in range(6):
        mean[k]=np.mean(data[k,:])
        if(k==0):
            data[k]=data[k]-mean[k]
        data[k]=data[k]-np.max(np.abs(data[k,:]))
        cc[k]=np.corrcoef(data[0,:],data[k,:])[0,1]
        
    
    content_cc="%s"%loc+"%.2f,%.2f,%.2f,%.2f,%.2f,%.2f"%(cc[0],cc[1],cc[2],cc[3],cc[4],cc[5])+"\n"
    content_mean="%s"%loc+"%.2f,%.2f,%.2f,%.2f,%.2f,%.2f"%(mean[0],mean[1],mean[2],mean[3],mean[4],mean[5])+"\n"
    #print(content_cc.strip())
    #print(content_mean.strip())
    file_dv.write(content_cc)
    file_mean.write(content_mean)
file_loc.close()
file_dv.close()
file_mean.close()